In [28]:
import warnings
warnings.filterwarnings('ignore')

import os
import json
import pandas as pd
import ipywidgets as widgets
from datetime import datetime
from ipywidgets import AppLayout, Button, Layout
from ipywidgets import Button, Layout, jslink, IntText, IntSlider
from IPython.display import display, Javascript, Markdown, clear_output, HTML
from ipywidgets import GridspecLayout
from markdown_it import MarkdownIt

In [29]:
%run ../02_ui_components/progressbar.ipynb

In [30]:
%run ../02_ui_components/sidebar.ipynb

In [ ]:
%run ../02_ui_components/header_button.ipynb

In [32]:
%run ../02_ui_components/recommendation_headers.ipynb

In [33]:
# Placeholder variables to store paths and data
# ques_nav_path = None
back_list = None
stored_choices = None
footer = None
path = None
back_path = None
df = None
mod = None
sd = None

In [34]:
# Declaring necessary variables and importing neccesary files for storing user answers and keep track of the visited pages
# global stored_choices, path, back_path, ques_nav_path, back_list

# stored_choices = {}
# path = "../data_files/json/temp_storage.json"
# back_path = "../data_files/json/backtracking.json"
dropdown_path = "../data_files/csv/links.csv"
ques_nav_path = "../data_files/json/ques_nav_list.json"

In [ ]:
# Creating button
def create_expanded_button(description, button_style):
    return Button(description=description, button_style=button_style, layout=Layout(height='auto', width='auto', border='1px solid black')) #width='max-content'

In [ ]:
def get_css_class(css_file, class_name):
    with open(css_file, 'r') as file:
        css = file.read()
        # Use regex to extract the class
        pattern = r'\.' + re.escape(class_name) + r'\s*\{([^}]*)\}'
        match = re.search(pattern, css, re.DOTALL)
        if match:
            css_content = match.group(1).strip()
            return f'\n<style>\n.{class_name} {{\n{css_content}\n}}\n</style>\n'
        else:
            return None

In [ ]:
css_path = "../css/style.css"

In [35]:
# Custom CSS to modify the question button design
ques_button_css = get_css_class(css_path, 'ques-button')

In [36]:
# Custom CSS to modify the option button design
opt_button_css = get_css_class(css_path, 'opt-button')

In [37]:
# Custom CSS to modify the recommendation button design
recom_button_css = get_css_class(css_path, 'recom-opt-button')

In [38]:
# Custom CSS to modify the recommendation button design
mod_header_button_css = get_css_class(css_path, 'mod-header-button')

In [39]:
# Custom CSS to modify the header design
header_css = get_css_class(css_path, 'custom-header')

In [40]:
# Custom CSS to modify the footer design
footer_css = get_css_class(css_path, 'custom-footer')

In [41]:
# Create an HTML widget for the background color
background_color_html = widgets.HTML(
    value = get_css_class(css_path, 'custom-vbox')
)

In [42]:
# Loading the header button and widget from other notebook
header = header_prog7
header_widget = header_widget

In [43]:
# widget to take the file name as input
file_name = widgets.Text(
    value='',
    placeholder='Paste your copied string here',
    description='String:',
    disabled=False   
)

In [44]:
# Create a buttons for submitting the file name
file_name_submit_button = widgets.Button(description="Submit", layout=widgets.Layout(border='1px solid black'))

In [45]:
# Creating a container for filename widget and button
header_right = widgets.VBox([file_name, file_name_submit_button], 
                    layout=widgets.Layout(width='auto', 
                                          padding_top= '10px', 
                                          align_items='flex-end',
                                          justify_content='center'))

In [46]:
footer_output = widgets.Output()
def on_file_button_clicked(event):
    with footer_output:
        clear_output()
        global ques_nav_path, back_list, stored_choices, path, back_path, footer, df, mod, sd, recom_button_css
        
        # reading the css files
        css_path = "../css/style.css"
        recom_button_css = get_css_class(css_path, 'recom-opt-button')
        
        # assigning the path varibales for each file
        main_file_name = file_name.value
        path = f"../data_files/json/{main_file_name}_storage.json"
        back_path = f"../data_files/json/{main_file_name}_back_path.json"
        ques_nav_path = f"../data_files/json/{main_file_name}_ques_nav_path.json"
        
        
        # creating storage files for each user
        if not os.path.isfile(path):
            with open(path, 'w') as file:
                json.dump({}, file)
        
        if not os.path.isfile(back_path):
            with open(back_path, 'w') as file:
                json.dump([], file)
                
        data_content = {
            "obj_list": [["Select a notebook", None]],
            "st_area_list": [["Select a notebook", None]],
            "dt_list": [["Select a notebook", None]],
            "ts_list": [["Select a notebook", None]],
            "ed_list": [["Select a notebook", None]],
            "da_list": [["Select a notebook", None]],
            "rec_list": [["Select a notebook", None]]
        }
        
        if not os.path.isfile(ques_nav_path):
            with open(ques_nav_path, 'w') as file:
                json.dump(data_content, file, indent=2)
          
        
        
        # reading form the files
        with open(path, "r") as f:
            stored_choices = json.load(f)
            
        with open(back_path, 'r') as f:
            back_list = json.load(f)
        
        df = pd.read_csv('../data_files/csv/analysis_consideration.csv', low_memory=False)
        mod = pd.read_csv('../data_files/csv/modelling_approach.csv', low_memory=False)
        sd = pd.read_csv('../data_files/csv/study_design_numeric.csv', low_memory=False)
        
        
        # generating dropdown notebook navigation bar and footer
        notebook_name = "test_recommendation-Copy"
        notebook_list_name = "rec_list"
        footer_name = "footer_in_prog7"
        
        footer = main(notebook_name, notebook_list_name, footer_name)
        
        # get suitable modelling approaches list and updated stored_choices
        mod_to_display_list, stored_choices = get_modelling_approach_list(mod, stored_choices)
        text_to_display_list = get_text_to_display_list(df, stored_choices)
        analysis_rec_container = get_analysis_rec_container(text_to_display_list, recom_button_css)
        sd_design_recommendation_dict = get_study_design_rec(sd, mod_to_display_list, stored_choices)
        final_sd_rec_widget_content_list_all_approach = get_final_sd_widget_content(mod_to_display_list, sd_design_recommendation_dict)
        
        display(footer)
file_name_submit_button.on_click(on_file_button_clicked)

In [47]:
# Creating a container for header image
header_container = widgets.HBox([header, header_widget, header_right], 
                    layout=widgets.Layout(width='95%',
                                          padding_top= '10px', 
                                          align_items='flex-start',
                                          justify_content='space-between'))

In [48]:
# FOR DISPLAYING MODELLING APPROACHES -------------------------->

def get_modelling_approach_list(mod, stored_choices):
    # For sorting the recommendation approaches
    original_mod_app_sequence = ["Species inventory", "Species diversity & richness", "Occupancy", "Relative abundance indices", "Behaviour"]

    mod_to_display_list = []

    mod_app_obj_condition = mod.loc[mod['approach_fulltext'] == 'Behaviour', 'obj_json'].iloc[0]
    mod_app_obj_condition_json = json.loads(mod_app_obj_condition)

    mod_app_cam_condition = mod.loc[mod['approach_fulltext'] == 'Behaviour', 'num_cams_json'].iloc[0]
    mod_app_cam_condition_json = json.loads(mod_app_cam_condition)

    if list(mod_app_obj_condition_json.keys())[0] in stored_choices.keys() and list(mod_app_cam_condition_json.keys())[0] in stored_choices.keys():
        if stored_choices[list(mod_app_obj_condition_json.keys())[0]] in mod_app_obj_condition_json[list(mod_app_obj_condition_json.keys())[0]] and stored_choices[list(mod_app_cam_condition_json.keys())[0]] > mod_app_cam_condition_json[list(mod_app_cam_condition_json.keys())[0]]:
            mod_to_display = "Behaviour"
        #     mod_to_display_dict["approach"].append(mod_to_display)
            mod_to_display_list.append(mod_to_display)

    mod_app_obj_condition = mod.loc[mod['approach_fulltext'] == 'Species diversity & richness', 'obj_json'].iloc[0]
    mod_app_obj_condition_json = json.loads(mod_app_obj_condition)

    mod_app_cam_condition = mod.loc[mod['approach_fulltext'] == 'Species diversity & richness', 'num_cams_json'].iloc[0]
    mod_app_cam_condition_json = json.loads(mod_app_cam_condition)

    mod_app_cam_inde_condition = mod.loc[mod['approach_fulltext'] == 'Species diversity & richness', 'cam_independent_json'].iloc[0]
    mod_app_cam_inde_condition_json = json.loads(mod_app_cam_inde_condition)

    if list(mod_app_obj_condition_json.keys())[0] in stored_choices.keys() and list(mod_app_cam_condition_json.keys())[0] in stored_choices.keys() and list(mod_app_cam_inde_condition_json.keys())[0] in stored_choices.keys():
        if stored_choices[list(mod_app_obj_condition_json.keys())[0]] in mod_app_obj_condition_json[list(mod_app_obj_condition_json.keys())[0]] and stored_choices[list(mod_app_cam_condition_json.keys())[0]] > mod_app_cam_condition_json[list(mod_app_cam_condition_json.keys())[0]] and stored_choices[list(mod_app_cam_inde_condition_json.keys())[0]] in mod_app_cam_inde_condition_json[list(mod_app_cam_inde_condition_json.keys())[0]]:
            mod_to_display = "Species diversity & richness"
            mod_to_display_list.append(mod_to_display)

    mod_app_obj_condition = mod.loc[mod['approach_fulltext'] == 'Species inventory', 'obj_json'].iloc[0]
    mod_app_obj_condition_json = json.loads(mod_app_obj_condition)

    mod_app_cam_condition = mod.loc[mod['approach_fulltext'] == 'Species inventory', 'num_cams_json'].iloc[0]
    mod_app_cam_condition_json = json.loads(mod_app_cam_condition)

    if list(mod_app_obj_condition_json.keys())[0] in stored_choices.keys() and list(mod_app_cam_condition_json.keys())[0] in stored_choices.keys():
        if stored_choices[list(mod_app_obj_condition_json.keys())[0]] in mod_app_obj_condition_json[list(mod_app_obj_condition_json.keys())[0]] and stored_choices[list(mod_app_cam_condition_json.keys())[0]] > mod_app_cam_condition_json[list(mod_app_cam_condition_json.keys())[0]]:
            mod_to_display = "Species inventory"
            mod_to_display_list.append(mod_to_display)

    mod_app_obj_condition = mod.loc[mod['approach_fulltext'] == 'Occupancy', 'obj_json'].iloc[0]
    mod_app_obj_condition_json = json.loads(mod_app_obj_condition)

    mod_app_cam_condition = mod.loc[mod['approach_fulltext'] == 'Occupancy', 'num_cams_json'].iloc[0]
    mod_app_cam_condition_json = json.loads(mod_app_cam_condition)

    mod_app_cam_inde_condition = mod.loc[mod['approach_fulltext'] == 'Occupancy', 'cam_independent_json'].iloc[0]
    mod_app_cam_inde_condition_json = json.loads(mod_app_cam_inde_condition)

    if list(mod_app_obj_condition_json.keys())[0] in stored_choices.keys() and list(mod_app_cam_condition_json.keys())[0] in stored_choices.keys() and list(mod_app_cam_inde_condition_json.keys())[0] in stored_choices.keys():
        if stored_choices[list(mod_app_obj_condition_json.keys())[0]] in mod_app_obj_condition_json[list(mod_app_obj_condition_json.keys())[0]] and stored_choices[list(mod_app_cam_condition_json.keys())[0]] > mod_app_cam_condition_json[list(mod_app_cam_condition_json.keys())[0]] and stored_choices[list(mod_app_cam_inde_condition_json.keys())[0]] in mod_app_cam_inde_condition_json[list(mod_app_cam_inde_condition_json.keys())[0]]:
            mod_to_display = "Occupancy"
            mod_to_display_list.append(mod_to_display)

    mod_app_obj_condition = mod.loc[mod['approach_fulltext'] == 'Relative abundance indices', 'obj_json'].iloc[0]
    mod_app_obj_condition_json = json.loads(mod_app_obj_condition)

    mod_app_cam_condition = mod.loc[mod['approach_fulltext'] == 'Relative abundance indices', 'num_cams_json'].iloc[0]
    mod_app_cam_condition_json = json.loads(mod_app_cam_condition)

    if list(mod_app_obj_condition_json.keys())[0] in stored_choices.keys() and list(mod_app_cam_condition_json.keys())[0] in stored_choices.keys():
        if stored_choices[list(mod_app_obj_condition_json.keys())[0]] in mod_app_obj_condition_json[list(mod_app_obj_condition_json.keys())[0]] and stored_choices[list(mod_app_cam_condition_json.keys())[0]] > mod_app_cam_condition_json[list(mod_app_cam_condition_json.keys())[0]]:
            mod_to_display = "Relative abundance indices"
            mod_to_display_list.append(mod_to_display)


    # # Sort mod_to_display_list according to the order in original_mod_app_sequence
    mod_to_display_list = sorted(mod_to_display_list, key=lambda x: original_mod_app_sequence.index(x))

    # FOR STORING MODELLING APPROACHES IN TEMP STORAGE ------------------->

    stored_choices["approach"] = mod_to_display_list
    with open(path, "w") as f:
        json.dump(stored_choices, f)

    with open(path, "r") as f:
        stored_choices = json.load(f)

    return mod_to_display_list, stored_choices

In [132]:
# FOR DISPLAYING ANALYSIS CONSIDERATIONS -------------------------->
def get_text_to_display_list(df, stored_choices):
    text_to_display_list = []
    condition = df.loc[df['field_code'] == 'sp_size', 'analysis_consideration_condition'].iloc[0]
    condition_json = json.loads(condition)
    if list(condition_json.keys())[0] in stored_choices:
        if stored_choices[list(condition_json.keys())[0]] in condition_json[list(condition_json.keys())[0]]:
            text_to_display = df.loc[df['field_code'] == 'sp_size', 'analysis_consideration_text'].iloc[0]
            text_to_display_list.append(text_to_display)

    # condition = df.loc[df['field_code'] == 'cam_strat_covar', 'analysis_consideration_condition'].iloc[0]
    # condition_json = json.loads(condition)
    # if list(condition_json.keys())[0] in stored_choices:
    #     if stored_choices[list(condition_json.keys())[0]] > condition_json[list(condition_json.keys())[0]]:
    #         text_to_display = df.loc[df['field_code'] == 'cam_strat_covar', 'analysis_consideration_text'].iloc[0]
    #         text_to_display_list.append(text_to_display)

    condition = df.loc[df['field_code'] == 'mod_rai', 'analysis_consideration_condition'].iloc[0]
    condition_json = json.loads(condition)
    if list(condition_json.keys())[0] in stored_choices:
        if stored_choices[list(condition_json.keys())[0]] in condition_json[list(condition_json.keys())[0]]:
            text_to_display = df.loc[df['field_code'] == 'mod_rai', 'analysis_consideration_text'].iloc[0]
            text_to_display_list.append(text_to_display)

    condition = df.loc[df['field_code'] == 'obj_targ_sp', 'analysis_consideration_condition'].iloc[0]
    condition_json = json.loads(condition)
    if list(condition_json.keys())[0] in stored_choices:
        if stored_choices[list(condition_json.keys())[0]] in condition_json[list(condition_json.keys())[0]]:
            text_to_display = df.loc[df['field_code'] == 'obj_targ_sp', 'analysis_consideration_text'].iloc[0]
            text_to_display_list.append(text_to_display)

    condition = df.loc[df['field_code'] == 'sp_detprob_cat', 'analysis_consideration_condition'].iloc[0]
    condition_json = json.loads(condition)
    if list(condition_json.keys())[0] in stored_choices:
        if stored_choices[list(condition_json.keys())[0]] in condition_json[list(condition_json.keys())[0]]:
            text_to_display = df.loc[df['field_code'] == 'sp_detprob_cat', 'analysis_consideration_text'].iloc[0]
            text_to_display_list.append(text_to_display)

    condition = df.loc[df['field_code'] == 'sp_rarity', 'analysis_consideration_condition'].iloc[0]
    condition_json = json.loads(condition)
    if list(condition_json.keys())[0] in stored_choices:
        if stored_choices[list(condition_json.keys())[0]] in condition_json[list(condition_json.keys())[0]]:
            text_to_display = df.loc[df['field_code'] == 'sp_rarity', 'analysis_consideration_text'].iloc[0]
            text_to_display_list.append(text_to_display)
            
    return text_to_display_list

In [134]:
# Creating button
def analysis_header_button(description, button_style):
    return Button(description=description, button_style=button_style, layout=Layout(height='80px', width='250px', border='1px solid black')) #width='max-content'

In [136]:
# IN PROGRESS--------------------FOR DISPLAYING ANALYSIS CONSIDERATIONS -------------------------->

def get_analysis_rec_container(text_to_display_list, recom_button_css):
    # analysis consideration grid
    analysis_header = analysis_header_button('Analysis consideration', 'success')
    analysis_header.style.button_color = '#FFA305'
    analysis_header.style.text_color = 'black'
    analysis_header.style.font_weight = 'bold'
    analysis_header.add_class('recom-opt-button') # Adding custom css properties to the side buttons

    # Define the CSS for the square widget
    analysis_widget_css = """
    <style>
    .analysis-widget {
        background-color: #FFBF52;
        color: black;
        font-size: 14px;
        display: flex;
        align-items: flex-start;
        justify-content: flex-start;
        flex-direction: column;
        text-align: flex-start;
        padding: 10px;
        box-sizing: border-box;
        border-radius: 10px;
    }
    </style>
    """

    # Create a HTML widget with the square widget CSS
    analysis_widget_html = widgets.HTML(value=analysis_widget_css)


    analysis_rec_placeholder = """We have no analysis considerations to recommend based on your selections!<br>However, 
    that doesn't mean that there aren't important aspects to consider. Please think through your design carefully to identify 
    potential sources of bias and error.<br><br>Here is an example of what you might see in this section:<br>
    <b>Because you chose...<em><br>State variable:</b> "Relative abundance" or "Unknown"</b></em><br><b>>>></b> 
    Note that when you reach the analysis stage, you will want to select a count model method that most appropriately accounts 
    for "Zero-inflation" and "Overdispersion" (see sections "Overdispersion/Zero-inflation [not yet present] and 
    "Mixed models" [not yet present]).."""



    # Initializing list to store analysis consideration widgets-----
    analysis_widgets_list = []


    # Initialize the content string
    if len(text_to_display_list) == 0:
        analysis_widget_content = """"""
        analysis_widget_content = f"""
        <div class="analysis-widget">
            <p>{analysis_rec_placeholder}</p>
        </div>
        """
        analysis_widget = widgets.HTML(value=analysis_widget_content, layout=Layout(max_width='700px', padding='5px'))
        analysis_widgets_list.append(analysis_widget)
    else:
        analysis_widget_content = """"""


    for text in text_to_display_list:
        analysis_widget_content = f"""
        <div class="analysis-widget">
            <p>{text}</p>
        </div>
        """
        analysis_widget = widgets.HTML(value=analysis_widget_content, layout=Layout(max_width='350px', padding='5px'))
        analysis_widgets_list.append(analysis_widget)


    # Creating HTML widget for displaying the custom added css classes
    analysis_header_widget = widgets.HTML(value=recom_button_css)



    first_hbox_container = widgets.HBox(analysis_widgets_list, layout=Layout(justify_content='flex-start', width='100%'))

    # Customize the layout of each widget and HBox
    hbox_container = widgets.HBox(children = [first_hbox_container, analysis_widget_html], layout=Layout(justify_content='flex-start', width='100%'))

    # Show the HBox container with custom layout
    analysis_rec_container = widgets.VBox(children = [analysis_header, analysis_header_widget, hbox_container], 
                                          layout=Layout(
                                              justify_content='flex-start', 
                                              width='100%',
                                              border= 'solid 1px black',
                                              padding= '10px'
                                          ))
    
    return analysis_rec_container

In [137]:
# Removes the comma and spaces and prepares a list of items
def split_and_trim(s):
    return [item.strip() for item in s.split(',')]

In [140]:
# TESTING WORK IN PROGRESS ----------------------------------->

def get_study_design_rec(sd, mod_to_display_list, stored_choices):
    # Initializing dictionary to store json condition for all the approaches
    sd_design_condition_dict = {}


    for mod_app in mod_to_display_list:
        sd_design_condition = sd.loc[(sd['approach_fulltext'] == mod_app) & (sd['user_answers'] == 'invalid'), 'json_logic'].iloc[0]
        sd_design_condition_json = json.loads(sd_design_condition)
        sd_design_condition_dict[mod_app] = sd_design_condition_json

    # Initializing dictionary to store recommendation for all the approaches
    sd_design_recommendation_dict = {}

    # Define the keys for the main dictionary and the nested keys
    modelling_approaches = mod_to_display_list
    nested_keys = ['cam_arrange', 'cam_spacing', 'num_cams', 'camdays_per_loc', 'cam_days_ttl', 'survey_duration']

    # Loop through the main keys to create the structure
    for key in modelling_approaches:
        sd_design_recommendation_dict[key] = {}
        for nested_key in nested_keys:
            # Assign a new list to each nested key
            sd_design_recommendation_dict[key][nested_key] = []


    for mod in mod_to_display_list:
        user_answers = ["invalid"]

        if mod == "Species inventory":
            if "sp_info" in stored_choices.keys():
                if stored_choices["sp_info"] in sd_design_condition_dict[mod]["sp_info"]:
                    user_answers.append("sp_info")
            if "sp_detprob_cat" in stored_choices.keys():
                if stored_choices["sp_detprob_cat"] in sd_design_condition_dict[mod]["sp_detprob_cat"]:
                    user_answers.append("sp_detprob_cat")
                else:
                    user_answers.append("sp_detprob_cat!")

        if mod == "Species diversity & richness":
            if "num_cams_avail" in stored_choices.keys():
                if stored_choices["num_cams_avail"] > sd_design_condition_dict[mod]["num_cams_avail"][0]:
                    user_answers.append("num_cams_avail")
            if "cam_arrange" in stored_choices.keys():
                if stored_choices["cam_arrange"] in sd_design_condition_dict[mod]["cam_arrange"]:
                    user_answers.append("cam_arrange")
            if "sp_asymptote" in stored_choices.keys():
                if stored_choices["sp_asymptote"] in sd_design_condition_dict[mod]["sp_asymptote"]:
                    user_answers.append("sp_asymptote")
            if "sp_size" in stored_choices.keys():
                if stored_choices["sp_size"] in sd_design_condition_dict[mod]["sp_size"]:
                    user_answers.append("sp_size")

        if mod == "Occupancy":
            if "data_hr" in stored_choices.keys():
                if stored_choices["data_hr"] == sd_design_condition_dict[mod]["data_hr"][0]:
                    user_answers.append("data_hr_true")
                elif stored_choices["data_hr"] == sd_design_condition_dict[mod]["data_hr"][1]:
                    user_answers.append("data_hr_false")
            if "num_cams_avail" in stored_choices.keys():
                if stored_choices["num_cams_avail"] > sd_design_condition_dict[mod]["num_cams_avail"][0]:
                    user_answers.append("num_cams_avail")
            if "sp_rarity" in stored_choices.keys():
                if stored_choices["sp_rarity"] == sd_design_condition_dict[mod]["sp_rarity"][0]:
                    user_answers.append("sp_rarity_rare")
                if stored_choices["sp_rarity"] == sd_design_condition_dict[mod]["sp_rarity"][1]:
                    user_answers.append("sp_rarity_less_common")
                if stored_choices["sp_rarity"] == sd_design_condition_dict[mod]["sp_rarity"][2]:
                    user_answers.append("sp_rarity_common")
            if "sp_detprob_cat" in stored_choices.keys():
                if stored_choices["sp_detprob_cat"] in sd_design_condition_dict[mod]["sp_detprob_cat"]:
                    user_answers.append("sp_detprob_cat")
                else:
                    user_answers.append("sp_detprob_cat!")
            if "sp_detprob_cat" in stored_choices.keys() and "sp_rarity" in stored_choices.keys():
                if stored_choices["sp_detprob_cat"] in sd_design_condition_dict[mod]["sp_detprob_cat"] and stored_choices["sp_rarity"] == sd_design_condition_dict[mod]["sp_rarity"][0]:
                        user_answers.append("sp_detprob_cat_sp_rarity")


        if mod == "Relative abundance indices":
            if "num_cams_avail" in stored_choices.keys():
                if stored_choices["num_cams_avail"] > sd_design_condition_dict[mod]["num_cams_avail"][0]:
                    user_answers.append("num_cams_avail")
            if "user_entry" in stored_choices.keys() and "num_det" in stored_choices.keys():
                if stored_choices["user_entry"] in sd_design_condition_dict[mod]["user_entry"] and stored_choices["num_det"] > sd_design_condition_dict[mod]["num_det"][0]:
                        user_answers.append("user_entry_num_det10")
                elif stored_choices["user_entry"] in sd_design_condition_dict[mod]["user_entry"] and stored_choices["num_det"] > sd_design_condition_dict[mod]["num_det"][1]:
                        user_answers.append("user_entry_num_det20")
            if "sp_rarity" in stored_choices.keys():
                if stored_choices["sp_rarity"] == sd_design_condition_dict[mod]["sp_rarity"][1]:
                    user_answers.append("sp_rarity_very_rare")
                if stored_choices["sp_rarity"] == sd_design_condition_dict[mod]["sp_rarity"][2]:
                    user_answers.append("sp_rarity_common")
            if "sp_type" in stored_choices.keys() and "sp_dense_low" in stored_choices.keys() and "sp_rarity" in stored_choices.keys():
                if (stored_choices["sp_type"] == sd_design_condition_dict[mod]["sp_type"][0] and stored_choices["sp_dense_low"] in sd_design_condition_dict[mod]["sp_dense_low"]) or (stored_choices["sp_type"] == sd_design_condition_dict[mod]["sp_type"][1] and stored_choices["sp_rarity"] == sd_design_condition_dict[mod]["sp_rarity"][0]):
                        user_answers.append("sp_type_sp_dens_low_sp_rarity")

        if mod == "Behaviour":
            if "data_hr" in stored_choices.keys():
                if stored_choices["data_hr"] == sd_design_condition_dict[mod]["data_hr"][0]:
                    user_answers.append("data_hr_true")
                elif stored_choices["data_hr"] == sd_design_condition_dict[mod]["data_hr"][1]:
                    user_answers.append("data_hr_false")
            if "cam_arrange" in stored_choices.keys() and "cam_strat_covar_num" in stored_choices.keys():
                if stored_choices["cam_arrange"] in sd_design_condition_dict[mod]["cam_arrange"] and stored_choices["cam_strat_covar_num"] > sd_design_condition_dict[mod]["cam_strat_covar_num"][0]:
                    user_answers.append("cam_arrange_cam_strat_covar_num")


        for user_answer in user_answers:
            for i in range(5, 11):
                temp_rec = sd.loc[(sd['approach_fulltext'] == mod) & (sd['user_answers'] == user_answer), sd.keys()[i]].iloc[0]
                if isinstance(temp_rec, str):
                    temp_rec = split_and_trim(temp_rec)
                    for item in temp_rec:
                        if item not in sd_design_recommendation_dict[mod][sd.keys()[i]]:
                            sd_design_recommendation_dict[mod][sd.keys()[i]].append(item)
        
        return sd_design_recommendation_dict

In [150]:
# WORK IN PROGRESS FOR MULTIPLE APPROACHES FOR STUDY DESIGN RECOMMENDATIONS -------------------------------->

def get_final_sd_widget_content(mod_to_display_list, sd_design_recommendation_dict):
    # study design recommendation grid
    sd_design_rec_header = create_expanded_button('Study design recommendation', 'success')
    sd_design_rec_header.style.button_color = '#FFA305'
    sd_design_rec_header.style.text_color = 'black'
    sd_design_rec_header.style.font_weight = 'bold'
    sd_design_rec_header.add_class('recom-opt-button') # Adding custom css properties to the side buttons

    # Define the CSS for the square widget
    sd_rec_widget_css = """
    <style>
    .sd-widget {
        white-space: normal;
        background-color: #FFBF52;
        color: black;
        font-size: 14px;
        display: flex;
        align-items: flex-start;
        justify-content: flex-start;
        flex-direction: column;
        text-align: flex-start;
        padding: 5px;
        box-sizing: border-box;
        border-radius: 10px;
    }
    </style>
    """

    # Create a HTML widget with the square widget CSS
    sd_rec_widget_html = widgets.HTML(value=sd_rec_widget_css)



    sd_rec_widget_content_list_all_approach = {}

    for mod in mod_to_display_list:
        # initializing conten string list
        sd_rec_widget_content_list = []

        # Initialize the content string
        for key in list(sd_design_recommendation_dict[mod].keys()):
        #     print(key)
            sd_rec_widget_content = """"""
            for text in sd_design_recommendation_dict[mod][key]:
                sd_rec_widget_content += f"""
                        <p style="margin: 5px 0; line-height: 1.2;"> {text} </p>
                    """
            sd_rec_widget_content_list.append(sd_rec_widget_content)
        sd_rec_widget_content_list_all_approach[mod] = sd_rec_widget_content_list



    final_sd_rec_widget_content_list_all_approach = {} 

    for mod in mod_to_display_list:  
        final_sd_rec_widget_content_list = []
        # Loop through the list and append each item to the content string
        for sd_content in sd_rec_widget_content_list_all_approach[mod]:
    #         print(type(sd_content))
            final_analysis_widget_content = f"""
                <div class="sd-widget">
                    <p>{sd_content}</p>
                </div>
                """
            final_sd_rec_widget_content_list.append(final_analysis_widget_content)
        final_sd_rec_widget_content_list_all_approach[mod] = final_sd_rec_widget_content_list
    
    return final_sd_rec_widget_content_list_all_approach

In [ ]:
#--------------CHEKCED SO FAR ----------------------

In [151]:
# # FOR STUDY DESIGN RECOMMENDATIONS -------------------------------->

# # study design recommendation grid
# sd_design_rec_header = create_expanded_button('Study design recommendation', 'success')
# sd_design_rec_header.style.button_color = '#FFA305'
# sd_design_rec_header.style.text_color = 'black'
# sd_design_rec_header.style.font_weight = 'bold'
# sd_design_rec_header.add_class('recom-opt-button') # Adding custom css properties to the side buttons

# # Define the CSS for the square widget
# sd_rec_widget_css = """
# <style>
# .sd-widget {
#     background-color: #FFBF52;
#     color: black;
#     display: flex;
#     align-items: flex-start;
#     justify-content: flex-start;
#     flex-direction: column;
#     text-align: flex-start;
#     padding: 10px;
#     box-sizing: border-box;
#     border-radius: 10px;
# }
# </style>
# """

# # Create a HTML widget with the square widget CSS
# sd_rec_widget_html = widgets.HTML(value=sd_rec_widget_css)


# # initializing conten string list
# sd_rec_widget_content_list = []

# # Initialize the content string
# for key in list(sd_design_rec_dict.keys()):
# #     print(key)
#     sd_rec_widget_content = """"""
#     for text in sd_design_rec_dict[key]:
#         sd_rec_widget_content += f"""
#                 <p> {text} </p>
#             """
#     sd_rec_widget_content_list.append(sd_rec_widget_content)
        
# final_sd_rec_widget_content_list = []        
        
        
# # Loop through the list and append each item to the content string
# for sd_content in sd_rec_widget_content_list:
#     final_analysis_widget_content = f"""
#         <div class="analysis-widget">
#             <p>{sd_content}</p>
#         </div>
#         """
#     final_sd_rec_widget_content_list.append(final_analysis_widget_content)

# sd_widget = widgets.HTML(value=final_sd_rec_widget_content_list[0])
# sd_widget2 = widgets.HTML(value=final_sd_rec_widget_content_list[1])
# sd_widget3 = widgets.HTML(value=final_sd_rec_widget_content_list[2])
# sd_widget4 = widgets.HTML(value=final_sd_rec_widget_content_list[3])
# sd_widget5 = widgets.HTML(value=final_sd_rec_widget_content_list[4])
# sd_widget6 = widgets.HTML(value=final_sd_rec_widget_content_list[5])

# #MAKE THIS LIST FOR ALL THE APPROACHES THEN CREATE ANOTHER LIST WITH THEM ----------------------------------------
# design_rec_final_widget_list = [sd_widget, sd_widget2, sd_widget3, sd_widget4, sd_widget5, sd_widget6]



# # Creating HTML widget for displaying the custom added css classes
# analysis_header_widget = widgets.HTML(value=recom_button_css)

# analysis_header_grid = GridspecLayout(7, 9, grid_gap='10px', height='550px', width='30%')
# analysis_header_grid[0:3, 0:8] = analysis_header
# analysis_header_grid[3, 0:8] = analysis_widget
# # analysis_header_grid[2, 1:7] = modelling_approach_2
# # analysis_header_grid[3, 1:7] = modelling_approach_3
# # analysis_header_grid[4, 1:7] = modelling_approach_4
# # analysis_header_grid[5, 1:7] = modelling_approach_5

In [152]:
# sd_design_rec_dict

In [153]:
# sd_rec_widget_content_list

In [154]:
# function for multiple modelling approaches

def mod_approaches(mod_app_name, design_rec_content):
    mod_app_header = create_expanded_button(mod_app_name, 'success')
    mod_app_header.style.button_color = '#71ADE5'
    mod_app_header.style.font_weight = 'bold'
    mod_app_header.style.text_color = 'black'
    mod_app_header.add_class('recom-opt-button')
    
    study_design_header = create_expanded_button("Study Design Recommendation", 'success')
    study_design_header.style.button_color = '#7EB738'
    study_design_header.style.font_weight = 'bold'
    study_design_header.style.text_color = 'black'
    study_design_header.add_class('recom-opt-button') # Adding custom css properties to the side buttons

    study_design_1 = create_expanded_button('Camera arrangement', 'success')
    study_design_1.style.button_color = '#B6D78E'
#     study_design_1.style.font_weight = 'bold'
    study_design_1.style.text_color = 'black'
    study_design_1.add_class('recom-opt-button')

    study_design_2 = create_expanded_button('Camera spacing', 'success')
    study_design_2.style.button_color = '#B6D78E'
#     study_design_2.style.font_weight = 'bold'
    study_design_2.style.text_color = 'black'
    study_design_2.add_class('recom-opt-button')

    study_design_3 = create_expanded_button('Number of cameras', 'success')
    study_design_3.style.button_color = '#B6D78E'
#     study_design_3.style.font_weight = 'bold'
    study_design_3.style.text_color = 'black'
    study_design_3.add_class('recom-opt-button')

    study_design_4 = create_expanded_button('Number of camera days per camera location', 'success')
    study_design_4.style.button_color = '#B6D78E'
#     study_design_4.style.font_weight = 'bold'
    study_design_4.style.text_color = 'black'
    study_design_4.add_class('recom-opt-button')

    study_design_5 = create_expanded_button('Total number of camera days', 'success')
    study_design_5.style.button_color = '#B6D78E'
#     study_design_5.style.font_weight = 'bold'
    study_design_5.style.text_color = 'black'
    study_design_5.add_class('recom-opt-button')

    study_design_6 = create_expanded_button('Survey duration', 'success')
    study_design_6.style.button_color = '#B6D78E'
#     study_design_6.style.font_weight = 'bold'
    study_design_6.style.text_color = 'black'
    study_design_6.add_class('recom-opt-button')

    # Creating HTML widget for displaying the custom added css classes
    study_design_widget = widgets.HTML(value=recom_button_css)

    sd_widget = widgets.HTML(value=design_rec_content[0])
    sd_widget2 = widgets.HTML(value=design_rec_content[1])
    sd_widget3 = widgets.HTML(value=design_rec_content[2])
    sd_widget4 = widgets.HTML(value=design_rec_content[3])
    sd_widget5 = widgets.HTML(value=design_rec_content[4])
    sd_widget6 = widgets.HTML(value=design_rec_content[5])

    

    study_design_grid = GridspecLayout(7, 25, grid_gap='10px', height='auto', width='auto')
    study_design_grid[0, 0:6] = mod_app_header
    study_design_grid[1, 0:6] = study_design_header

    study_design_grid[2, 0:4] = study_design_1
    study_design_grid[3:7, 0:4] = sd_widget

    study_design_grid[2, 4:8] = study_design_2
    study_design_grid[3:7, 4:8] = sd_widget2

    study_design_grid[2, 8:12] = study_design_3
    study_design_grid[3:7, 8:12] = sd_widget3

    study_design_grid[2, 12:16] = study_design_4
    study_design_grid[3:7, 12:16] = sd_widget4

    study_design_grid[2, 16:20] = study_design_5
    study_design_grid[3:7, 16:20] = sd_widget5

    study_design_grid[2, 20:24] = study_design_6
    study_design_grid[3:7, 20:24] = sd_widget6
    
    return study_design_grid

In [155]:
# grid = mod_approaches("Occupancy Model")

In [163]:
# mod_to_display_list
mod_app_container_list = []

In [164]:
for mod in mod_to_display_list:
    mod_app_container = mod_approaches(mod, final_sd_rec_widget_content_list_all_approach[mod])
    temp_container = widgets.VBox(children = [mod_app_container], layout=widgets.Layout(
#                 padding='10px'
#                 border='solid 2px black'
            ))
    mod_app_container_list.append(temp_container)

In [165]:
# Define the CSS for the square widget
default_modrec_widget_css = """
<style>
.dmodrec-widget {
    background-color: #51DB86;
    color: black;
    font-size: 15px !important;
    display: flex;
    align-items: center;
    justify-content: center;
    flex-direction: column;
    text-align: flex-start;
    padding: 10px;
    box-sizing: border-box;
    border-radius: 10px;
}
</style>
"""

# Create a HTML widget with the square widget CSS
def_mod_rec_widget_html = widgets.HTML(value=default_modrec_widget_css)


def_mod_rec_placeholder = """Based on your selections, none of the modeling approaches were appropriate, 
please review your selections to consider less restrictive options.<br>You can also refer to 
<a href="https://ab-rcsc.github.io/RCSC-WildCAM_Remote-Camera-Survey-Guidelines-and-Metadata-Standards/1_survey-guidelines/1_10.1_AppendixA-Tables.html" target="_blank"
style="color: blue; text-decoration: underline;">
[Appendix A - Table A2]
</a> of the Remote Camera Survey Guidelines (RCSC et al., 2024) to determine what might be too restrictive 
(e.g., if you only have 10 cameras, you will see that few approaches are appropriate)."""


# Initialize the content string
if len(mod_to_display_list) == 0:
    def_mod_rec_widget_content = """"""
    def_mod_rec_widget_content = f"""
    <div class="dmodrec-widget">
        <p>{def_mod_rec_placeholder}</p>
    </div>
    """
    def_mod_rec_widget = widgets.HTML(value=def_mod_rec_widget_content, layout=Layout(max_width='700px', padding='5px'))
    mod_app_container_list.append(def_mod_rec_widget)
    mod_app_container_list.append(def_mod_rec_widget_html)

In [171]:
main_container = widgets.VBox(mod_app_container_list, layout=widgets.Layout(
#         margin='auto',  
#         align_items='flex-start',  
        justify_content='flex-start'
            ))

In [172]:
# main_container

In [95]:
# # Creating a container to store the factor widget and added custom css widgets
# demo_grid_container = widgets.HBox(children=[study_design_grid, modelling_approach_grid, analysis_header_grid, modelling_approach_widget, study_design_widget, analysis_header_widget, analysis_widget_html], layout=widgets.Layout(
#     align_items='center',  # Center align vertically
#     margin = '20px',
#     overflow= 'auto',
# #     border='solid 2px black',
# #     width = "70%"
# #     justify_content='center'  # Center align horizontally
# #     height='80vh'  # Set height to 100% of viewport height for vertical centering
# ))

In [96]:
# Creating a container to store the factor widget and added custom css widgets
# , analysis_widget_html, analysis_header_grid
demo_container = widgets.VBox(children=[main_container, study_design_widget, sd_rec_widget_html], layout=widgets.Layout(
    align_items='flex-start',  # Center align vertically
#     margin = 'auto',
    padding='10px',
    min_height = "580px",
#     height = "auto",
    overflow= 'auto',
#     overflow_y='scroll',
    border='solid 1px black',
    min_width = "100%"
#     justify_content='center'  # Center align horizontally
#     height='80vh'  # Set height to 100% of viewport height for vertical centering
))

# demo_container.add_class("overflow-y: 'auto';")

In [97]:
# Creating a container to store the factor widget and added custom css widgets
demo_grid_container = widgets.VBox(children=[mod_app_header_container, background_color_html, demo_container, analysis_rec_container], layout=widgets.Layout(
        width='99%',  
        height='auto', 
        padding='10px',
        margin='auto',  
        align_items='center',  
        justify_content='center'
))

In [98]:
# Create a button for going back to previous question
back_button = widgets.Button(description="<< Back", layout=widgets.Layout(border='1px solid black'))

In [99]:
# For going back to last question

output = widgets.Output()
def on_back_button_clicked(event):
    with output:
        clear_output()
        file_name = back_list[-1]
        notebook_filename = file_name + ".ipynb"
        js_code = f"window.open('{notebook_filename}', '_self');"
#         link = Markdown('''<a href="{}" target="_self" rel="noopener noreferrer">Previous</a>'''.format(notebook_filename))
        back_list.pop()
        with open (back_path, "w")as f:
            json.dump(back_list, f)
        display(Javascript(js_code))

back_button.on_click(on_back_button_clicked)

In [ ]:


# place an output widget here after the hedaer_container, then declare that in the onclick function for file_name input.
# insert everything inside the onclick function

demo_right_col = widgets.VBox(children=[header_container, background_color_html], layout=widgets.Layout(
        width='88%',  
        height='90vh',  
        border_left='solid 1px grey',
        padding='5px',
        margin='auto',  
        align_items='center',  
        justify_content='flex-start'
))

In [100]:
# Creating containers for all the widgets along with their css properties

all_button = widgets.HBox(
    [back_button],
    layout=widgets.Layout(
        width='99%',  
        height='auto',
#         border='solid 2px grey',
#         padding='1px',
        margin='auto',  
        align_items='center',  
        justify_content='space-between' 
    )
)
buttons = widgets.VBox(
    [output, all_button],
    layout=widgets.Layout(
        width='99%',  
        height='auto',  
#         border='solid 2px grey',
#         padding='1px',
        margin='auto',  
        align_items='center',  
        justify_content='center'  
    )
)



upper = widgets.VBox(
    [demo_grid_container, buttons],
    layout=widgets.Layout(
        width='95%',  
        height='auto', 
        border='solid 1px grey',
        padding='10px',
        margin='auto',  
        align_items='center',  
        justify_content='center' 
    )
)
upper.add_class("custom-vbox")

right_col = widgets.VBox(children=[header_container, background_color_html, upper, footer], layout=widgets.Layout(
        width='88%',  
        height='auto',  
        border_left='solid 1px grey',
        padding='5px',
        margin='auto',  
        align_items='center',  
        justify_content='center'
))

# cols = widgets.HBox(
#     [left_col, right_col],
#     layout=widgets.Layout(
#         width='95%',  
#         height='auto',  
#         border='solid 2px grey',
# #         padding='10px',
#         margin='auto',  
#         align_items='flex-start', 
#         justify_content='center'
#     )
# )


cols = widgets.HBox(
    [left_col, demo_right_col],
    layout=widgets.Layout(
        width='95%',  
        height='auto',  
        border='solid 2px grey',
#         padding='10px',
        margin='auto',  
        align_items='flex-start', 
        justify_content='center'
    )
)
display(cols)

In [70]:
# # Function to save session data as JSON with a timestamped filename
# def save_json_with_timestamp(data, folder='final_session_data'):
#     # Create folder if it doesn't exist
#     if not os.path.exists(folder):
#         os.makedirs(folder)
    
#     # Get the current timestamp and format it
#     timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")  # e.g., 20230912_134500
#     filename = f"session_info_{timestamp}.json"

#     # Full path to save the file
#     file_path = os.path.join(folder, filename)
    
#     # Save the data as a JSON file
#     with open(file_path, 'w') as file:
#         json.dump(data, file, indent=4)  # 'indent=4' makes the JSON file more readable

# # Save the session data as a JSON file
# save_json_with_timestamp(stored_choices)

JSON data saved to session_data\session_info_20240912_023638.json


In [71]:
# with open(path, 'w') as file:
#     json.dump({}, file, indent=4)

In [ ]:
# with open(back_path, 'w') as file:
#     json.dump({}, file, indent=4)